In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from bertopic import BERTopic

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [5]:
# Load sample dataset
newsgroups_data = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
documents = newsgroups_data.data[:500]  # Limit to 500 documents for faster computation

/opt/anaconda3/lib/python3.12/site-packages/sklearn/datasets/_twenty_newsgroups.py:79: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tarfile.open(archive_path, "r:gz").extractall(path=target_dir)


In [6]:
df = pd.DataFrame(documents, columns=["text"])

## Comparing Sentiment

### Vader

In [7]:
# Initialize VADER Sentiment Analyzer
vader_analyzer = SentimentIntensityAnalyzer()
df["vaderSent"] = df.text.apply(lambda x: vader_analyzer.polarity_scores(x))

### BERT

In [8]:
df["trunc_text"] = df.text.apply(lambda x: x[:500])

In [9]:
bert_classifier = pipeline('sentiment-analysis')
bert_scores = bert_classifier(df.trunc_text.tolist())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use mps:0
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
/opt/anaco

KeyboardInterrupt: 

In [7]:
df["bertSent"] =  bert_scores

### Check the correlation between the two of these sentiment analyses pipelines

In [ ]:
# your code

Find an examples where the two sentiment classifications are opposing. Why might this be the case?

# Topic Modeling

### Latent Dirichlet Allocation

In [10]:
num_topics = 10

In [11]:
list_stopwords = stopwords.words("english")
porter = PorterStemmer()

def process_step(one_str):   
    nostop_listing = [word for word in wordpunct_tokenize(one_str)
                      if word not in list_stopwords]
    clean_listing = [porter.stem(word) for word in nostop_listing
                    if word.isalpha() 
                    and len(word) > 3]
    clean_listing_str = " ".join(clean_listing)
    return(clean_listing_str)
    
df["text_proc"] = df.text.apply(process_step)

In [12]:
# Preprocess data
documents_clean = [gensim.utils.simple_preprocess(doc) for doc in df.text_proc]
dictionary = corpora.Dictionary(documents_clean)
corpus = [dictionary.doc2bow(doc) for doc in documents_clean]

In [13]:
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, random_state=42)

In [14]:
topics = lda_model.print_topics(num_words = 10)

for topic in topics:
    print(topic)


(0, '0.009*"would" + 0.004*"year" + 0.004*"also" + 0.004*"know" + 0.003*"think" + 0.003*"includ" + 0.003*"they" + 0.003*"right" + 0.003*"time" + 0.003*"mean"')
(1, '0.007*"would" + 0.005*"peopl" + 0.004*"use" + 0.004*"well" + 0.004*"argument" + 0.004*"like" + 0.004*"thi" + 0.004*"know" + 0.003*"scsi" + 0.003*"look"')
(2, '0.006*"would" + 0.005*"like" + 0.005*"peopl" + 0.004*"armenian" + 0.004*"time" + 0.003*"system" + 0.003*"think" + 0.003*"use" + 0.003*"know" + 0.003*"make"')
(3, '0.005*"use" + 0.004*"know" + 0.004*"would" + 0.003*"thi" + 0.003*"find" + 0.003*"come" + 0.003*"time" + 0.003*"father" + 0.003*"want" + 0.003*"window"')
(4, '0.004*"year" + 0.004*"know" + 0.004*"want" + 0.004*"would" + 0.004*"use" + 0.003*"time" + 0.003*"like" + 0.003*"thing" + 0.003*"window" + 0.003*"even"')
(5, '0.006*"would" + 0.005*"think" + 0.004*"know" + 0.004*"peopl" + 0.004*"like" + 0.004*"much" + 0.004*"thing" + 0.004*"thi" + 0.003*"true" + 0.003*"armenian"')
(6, '0.006*"use" + 0.005*"anyon" + 0.004

In [15]:
### visualize
lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=94291) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=94291) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=94291) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=94291) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin

KeyboardInterrupt: 

## BERTopic

In [16]:
documents = newsgroups_data.data[:2000]

In [17]:
%%time
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 41.5 s, sys: 7.6 s, total: 49.1 s
Wall time: 6min 1s


In [18]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,645,-1_the_to_of_and,"[the, to, of, and, is, for, it, in, that, you]","[Hi Netters,\n\nAs promised, here are the summ..."
1,0,191,0_he_the_in_team,"[he, the, in, team, to, and, game, was, play, ...",[1992-93 Los Angeles Kings notes.\n-----------...
2,1,144,1_is_of_the_that,"[is, of, the, that, to, not, and, jesus, it, in]","[To what follows, our moderator has already an..."
3,2,99,2_dos_for_good_excellent,"[dos, for, good, excellent, offer, 150, shippi...",[\n\n What phony names? My name is clearly ...
4,3,87,3_of_it_the_is,"[of, it, the, is, to, in, that, and, are, you]",[\nAll true. And all good points.\n\n\n\nWell...
5,4,64,4_israel_the_of_to,"[israel, the, of, to, and, in, that, you, is, by]",[\n CHECK MENAHEM BEGIN DAIRIES (published b...
6,5,62,5_deletion_huh_david_,"[deletion, huh, david, , , , , , , ]","[David\n\n\n, \nHuh?, \n(Deletion)\n ]"
7,6,54,6_space_the_and_nasa,"[space, the, and, nasa, of, for, to, shuttle, ...",[Archive-name: space/new_probes\nLast-modified...
8,7,52,7_gun_the_of_to,"[gun, the, of, to, that, and, in, is, be, was]",[\nThe Supreme Court seems to disagree with yo...
9,8,51,8_to_the_clipper_encryption,"[to, the, clipper, encryption, chip, key, be, ...",[\n > Clipper might be a good way to cover th...


## Your turn

#### Try running BERTopic on the prcessed data (rather than the uncleaned one)
#### What do you find?

In [21]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents_clean)

/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


AttributeError: 'list' object has no attribute 'astype'